*This notebook is dedicated to scraping all images from a Pinterest board and saving them locally. One board typically contains between 700-800 images. For this project, I scraped over 10,000 images in order to train my autoencoder. I reused this code for several different boards. Just place a different link for the url variable and change the name of the files being saved.*

In [ ]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
url = 'https://www.pinterest.com/{your search query here}'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

*In order to use the Chromedriver successfully, you must have a Pinterest account and log in after launching the driver*

*Because Pinterest uses Javascript to load images as you scroll, for this to work, the code scrolls down the page a bit, takes all the image hyperlink references available, and appends them into a list. Naturally, this method produces a lot of duplicates hrefs. This is dealt with at the end by converting the list into a set.*

In [ ]:
srcs = []
for i in range(0,300):
    driver.execute_script(f'window.scrollTo({i*400},{i*400+400})')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    imgs = soup.find(class_='gridCentered').find_all('img')
    for img in imgs:
        srcs.append(img['src'])
urls = set(srcs)

In [ ]:
driver.quit()

*With all the hrefs, its time to save all the images. Most are saved as low quality, small images in order to load quicker during user experience. Fortunately, Pinterest has a uniform format for all its images, and the quality can be easily improved by substituting the 236x (236 pixels) or 564x with 1200x to get the full volume image for your files.*

In [ ]:
for i, url in enumerate(list(urls)[528:], 528):
    url = re.sub('\d+x', '1200x', url)
    response = requests.get(url)
    file = open(f"more_pinterest/pinterest_your_name_here_{i}.jpeg", "wb")
    file.write(response.content)
    file.close()